In [ ]:
%load_ext autoreload
%autoreload 2
from galsbi import GalSBI
from cosmic_toolbox import arraytools as at, colors
from trianglechain import TriangleChain
import matplotlib.pyplot as plt

colors.set_cycle()

# Default behavior

Initialize the model with the configurations of choice.

In [ ]:
model = GalSBI("Fischbacher+24")

Generate an intrinsic catalog from this model. If you run a model for the first time, it might take a bit longer since some files have to be loaded to cache.
By default, the saved catalogs only have the intrinsic parameters and are sampled from the first healpy pixels of a map with nside 64.

In [ ]:
model()

Then, the catalogs can be loaded for all bands and you can plot the intrinsic distribution. The name of the catalogs are `{catalog_name}_{model_index}_{band}_ucat.gal.cat` and `{catalog_name}_{model_index}_{band}_ucat.star.cat`. The catalog name and the model index can easily be changed as an argument of the called model. The bands are fixed for a given model. To add other bands, writing a custom config file is needed. You can directly load the catalogs using the `load_catalogs` module. It returns a dictionary with all the catalogs written in the step above. The format of the catalog can be a structured numpy array (`rec`), a pandas dataframe (`df`) or a fits table (`fits`).

In [ ]:
cats = model.load_catalogs(output_format="rec")
cat = cats["ucat galaxies i"]

ranges = {
    "mag": [18, 27],
    "r50": [0, 4],
    "sersic_n": [0, 5],
    "z": [0, 6]
}
tri = TriangleChain(ranges=ranges, params=list(ranges.keys()), fill=True)
tri.contour_cl(cat);

If you want to have one combined catalog with information from all bands, you can load it with the `combine=True` parameter.

In [ ]:
cats = model.load_catalogs(combine=True)
cat = cats["ucat galaxies"]

ranges = {
    "mag g": [18, 27],
    "mag r": [18, 27],
    "mag i": [18, 27],
    "mag z": [18, 27],
    "mag y": [18, 27],
    "r50": [0, 4],
    "sersic_n": [0, 5],
    "z": [0, 6]
}
tri = TriangleChain(ranges=ranges, params=list(ranges.keys()), fill=True)
tri.contour_cl(cat);

All the galaxies are positioned by default in the first healpy index.

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u

def plot_ra_dec(ra, dec):
    coords = SkyCoord(cat["ra"], cat["dec"], unit=(u.deg, u.deg))

    plt.figure(figsize=(10, 5))
    ax = plt.subplot(111, projection='mollweide')

    sc = ax.scatter(coords.ra.radian, coords.dec.radian, s=1)

    plt.title('RA vs Dec Mollweide Projection')
    ax.set_xlabel('RA (radians)')
    ax.set_ylabel('Dec (radians)')

    ax.grid(True)

plot_ra_dec(cat["ra"], cat["dec"]);

Find out which papers you should cite with the configuration you have chosen above.

In [ ]:
model.cite()

# Specify survey size

The easiest way to specify a specific survey area where a galaxy catalog should be created is by passing a boolean healpy map to the `GalSBI` class.

In [ ]:
import healpy as hp
import numpy as np
import matplotlib.pyplot as plt

nside = 2048
npix = hp.nside2npix(nside)

mask = np.zeros(npix)

# Define a circular patch around the point (theta, phi) = (45 degrees, 45 degrees)
theta_center = np.radians(45.0)
phi_center = np.radians(45.0)

# Set a 0.5 degree radius
radius = np.radians(0.5)

# Find all pixel indices within this patch and set mask to 1
vec_center = hp.ang2vec(theta_center, phi_center)
patch_pixels = hp.query_disc(nside, vec_center, radius)
mask[patch_pixels] = 1

In [ ]:
model = GalSBI("Fischbacher+24")
model(healpix_map=mask)

As expected, the catalog looks the same since the galaxies are drawn from the same galaxy population model but the positions of the galaxies are located at the area specified by the map defined above.

In [ ]:
cats = model.load_catalogs(output_format="rec")
cat = cats["ucat galaxies i"]

ranges = {
    "mag": [18, 28],
    "r50": [0, 4],
    "sersic_n": [0, 5],
    "z": [0, 6]
}
tri = TriangleChain(ranges=ranges, params=list(ranges.keys()), fill=True)
tri.contour_cl(cat);

plot_ra_dec(cat["ra"], cat["dec"])

For larger survey areas, sampling galaxies in one call might become infeasible due to the additional memory and time that is required. However, the catalog generation can easily be split into smaller chunks which reduces the memory consumption significantly. Below we show an example where we split the mask into many submask that sample only one healpix pixels each. Speeding up the catalog generation can be achieved by parallelizing the last for-loop of this script, using e.g. `esub-epipe` to submit each index as a separate job on a cluster or classic parallelization tools such as Python's `multiprocessing` module or `concurrent.futures`.

In [ ]:
def process_healpix_submask(submask, catalog_name):
    model = GalSBI("Moser+24")
    model(healpix_map=submask, file_name=catalog_name)

submasks = []
catalog_names = []
pixels_per_mask = 10
n_pixels = len(patch_pixels)
for i in range(n_pixels // pixels_per_mask + 1):
    submask = np.zeros(npix)
    pixels = patch_pixels[i*pixels_per_mask : (i+1)*pixels_per_mask]
    submask[pixels] = 1
    submasks.append(submask)
    catalog_names.append(f"catalogs_pixelbatch{i}")
    
for i in range(len(submasks)):
    process_healpix_submask(submasks[i], catalog_name=catalog_names[i])

# Emulating image simulations

The above examples generate intrinsic catalogs. Not all of these galaxies would actually be detected when building up a catalog from actual astronomical images. The **detection probability** is to first order a function of the apparent magnitude. However, there are further effects such as **blending** or **background noise** that impact the detection probability (especially at the faint edge). Furthermore, an intrinsic size or magnitude will not the be the size that is measured in an actual astronomical image by a source extraction software such as `SExtractor`. Both background noise or the point spread function (**PSF**) have a strong impact on the measured parameters and introduce a scatter.

Ideally, these effects are taken into account by performing image simulations (see below). But as shown in Fischbacher et al. (2024), it is possible to emulate this effects accurately. We show below how to generate a galaxy catalog with an emulator trained on HSC deep fields. Note that the detection classifiers predicts if an object is detected as a galaxy, most stars are therefore are rejected and only a few stars contaminate the catalog (based on the accuracy of the star galaxy separation method).

In [ ]:
model = GalSBI("Fischbacher+24")
model(mode="emulator")

Two catalogs are generated. The catalog of all intrinsic objects are saved as `{file_name}_{model_index}_{band}_ucat.gal.cat` and `{file_name}_{model_index}_{band}_ucat.star.cat`. The catalog of the measured objects is called `{file_name}_{model_index}_{band}_se.cat`.

In [ ]:
cats = model.load_catalogs(output_format="rec")

cat_ucat = cats["ucat galaxies i"]
cat_sextractor = cats["sextractor i"]

ranges = {
    "mag": [18, 28],
    "r50": [0, 4],
    "sersic_n": [0, 5],
    "z": [0, 6],
    "MAG_AUTO": [18, 28],
    "FLUX_RADIUS": [0, 10],
    "log10_snr": [0, 5],
}
tri = TriangleChain(ranges=ranges, params=list(ranges.keys()), fill=True, histograms_1D_density=False)
tri.contour_cl(cat_ucat, label="intrinsic");
tri.contour_cl(cat_sextractor, label="measured", show_legend=True);

# Create an image

Running a realistic image simulation to compare with actual data (as it is done while constraining the galaxy population) requires a lot of work to correctly model the background and PSF for all real images. We give an example how to generate a simple image with constant PSF and background level. Note that the location of the image is not defined via a healpy map anymore but by defining the center of the image, the pixel scale and the number of pixels. We choose values for the image systematics that are represantative for HSC deep fields.

In [ ]:
model = GalSBI("Fischbacher+24")
model(mode="image")

The image is save under `{file_name}_{model_index}_{band}_image.fits` but can also be loaded using `load_image` directly

In [ ]:
from astropy.visualization import ImageNormalize, LogStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization import ZScaleInterval, PercentileInterval
import matplotlib.pyplot as plt

images = model.load_images()
image = images["image i"]

interval = PercentileInterval(99)
vmin, vmax = interval.get_limits(image)
norm = ImageNormalize(vmin=vmin, vmax=vmax)

plt.imshow(image, cmap='gray', norm=norm)

# Create an image and source extraction

After simulating the image, it is also possible to run `SExtractor` on the image. You then receive a catalog of intrinisic and measured quantities similar to the emulator case alongside the image and the background and segmentation map by `SExtractor`.

In [ ]:
model = GalSBI("Fischbacher+24")
model(mode="image+SE")

In [ ]:
images = model.load_images()
image = images["image i"]
bkg = images["background i"]
seg = images["segmentation i"]

interval = PercentileInterval(95)
vmin, vmax = interval.get_limits(image)
norm = ImageNormalize(vmin=vmin, vmax=vmax)

fig, axs = plt.subplots(1, 3, figsize=(9,3))
axs[0].imshow(image, cmap='gray', norm=norm)
axs[1].imshow(bkg)
axs[2].imshow(seg!=0, cmap="Blues")

In [ ]:
cats = model.load_catalogs()
cat_ucat = cats["ucat galaxies i"]
cat_sextractor = cats["sextractor i"]

de_kwargs = {
    "smoothing_parameter1D": 0.5,
    "smoothing_parameter2D": 0.5,
}

ranges = {
    "mag": [18, 28],
    "r50": [0, 4],
    "sersic_n": [0, 5],
    "z": [0, 6],
    "MAG_AUTO": [18, 28],
    "FLUX_RADIUS": [0, 10],
}
tri = TriangleChain(ranges=ranges, params=list(ranges.keys()), fill=True, histograms_1D_density=False, de_kwargs=de_kwargs)
tri.contour_cl(cat_ucat, label="intrinsic");
tri.contour_cl(cat_sextractor, label="measured", show_legend=True);

# Custom config file

`galsbi` is highly customizable. An overview of parameters that can be passed to `ucat` or `ufig` can be found in the corresponding common files. Each of these parameters can either be passed as an argument to the call of the model or a completely new config file can be passed. We advice to use the existing config files of the corresponding galaxy population models as a template. In this config files it is clearly stated which parameters should not be changed when using this galaxy population model (e.g. the parametrization of the luminosity function because this changes the meaning of the parameters of the galaxy population model).

The config file can be passed as a path or a module. Below we show an example where we load the basic intrinsic config file using both versions.

In [ ]:
import galsbi
import os


path2config = os.path.join(galsbi.__path__[0], "configs/config_Fischbacher+24_intrinsic.py")
model = GalSBI("Fischbacher+24")
model(mode="config_file", config_file=path2config)

config_module = "galsbi.configs.config_Fischbacher+24_intrinsic"
model = GalSBI("Fischbacher+24")
model(mode="config_file", config_file=config_module)